# connect

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/BKT

/content/gdrive/MyDrive/BKT


#Explore data
## Removed few columns from Assistment dataset which aren't necessary

In [4]:
import numpy as np
import pandas as pd
import csv
from csv import reader

import re
import sys
import argparse

import math
import operator

from sklearn import metrics
from sklearn.metrics import roc_auc_score



In [ ]:
df = pd.read_csv('sbnew.csv', encoding = 'latin', low_memory = False)
print(df.columns)
df

Index(['user_id', 'problem_id', 'correct', 'attempt_count', 'answer_type',
       'type', 'skill_id', 'skill_name'],
      dtype='object')


,user_id,problem_id,correct,attempt_count,answer_type,type,skill_id,skill_name
0,64525,51424,1,1,algebra,MasterySection,1.0,Box and Whisker
1,64525,51435,1,1,algebra,MasterySection,1.0,Box and Whisker
2,70363,51444,0,2,algebra,MasterySection,1.0,Box and Whisker
3,70363,51395,1,1,algebra,MasterySection,1.0,Box and Whisker
4,70363,51481,0,14,algebra,MasterySection,1.0,Box and Whisker
...,...,...,...,...,...,...,...,...
525529,96299,108976,1,1,choose_1,MasterySection,NaN,NaN
525530,96299,109015,1,1,choose_1,MasterySection,NaN,NaN
525531,96299,54060,1,1,algebra,MasterySection,NaN,NaN
525532,96299,54061,1,1,algebra,MasterySection,NaN,NaN


# Sort the data using student id and save as csv

In [ ]:
sample = open('sbnew.csv','r')
csv1 = csv.reader(sample)

sort = sorted(csv1,key = operator.itemgetter(0))


In [ ]:
sort

In [ ]:
# saving as a csv file
file = open('sorted_student.csv', 'w+', newline ='')
with file:    
    write = csv.writer(file)
    write.writerows(sort)

In [ ]:
df1 = pd.read_csv('sorted_student.csv', encoding = 'latin', low_memory = False)
print(df1.columns)
df1

Index(['user_id', 'problem_id', 'correct', 'attempt_count', 'answer_type',
       'type', 'skill_id', 'skill_name'],
      dtype='object')


,user_id,problem_id,correct,attempt_count,answer_type,type,skill_id,skill_name
0,14,93383,0,1,algebra,MasterySection,2.0,Circle Graph
1,14,93407,1,1,algebra,MasterySection,2.0,Circle Graph
2,14,93400,0,1,algebra,MasterySection,2.0,Circle Graph
3,14,93419,0,1,algebra,MasterySection,2.0,Circle Graph
4,14,93420,0,1,algebra,MasterySection,2.0,Circle Graph
...,...,...,...,...,...,...,...,...
525529,96299,108976,1,1,choose_1,MasterySection,NaN,NaN
525530,96299,109015,1,1,choose_1,MasterySection,NaN,NaN
525531,96299,54060,1,1,algebra,MasterySection,NaN,NaN
525532,96299,54061,1,1,algebra,MasterySection,NaN,NaN


# Creating a list of lists, Each sublist contains data about a single student



In [5]:
unique_students = [] # Total No. of unique students
student_data = [] # list of lists where each sublist contains data about a unique student
single_student_data = []

with open('sorted_student.csv') as obj:

  file_reader = reader(obj)
  # Not considering first line of csv, cuz its just column names
  header_line = next(file_reader) 
  if header_line != None: 
    for row in file_reader: 

      # First row in the csv file contains student ids 
      student_id = row[0]

      # Gets true for each unique student
      if not student_id in unique_students:

        # Gets added to the list of unique student data
        unique_students.append(student_id)

        if len(single_student_data) > 0:
          student_data.append(single_student_data)
          single_student_data = [] 

      # Holds all the data of a unique user
      single_student_data.append(row)

       
    if len(single_student_data)> 0:
      student_data.append(single_student_data)        
            
            

In [6]:
len(unique_students) 

4217

# Create problem id-skills mapping 'List'

## Sort the data with problem ids and make a csv

In [ ]:
unique_problems = []

In [ ]:
# Appending all unique problems
with open('sorted_student.csv') as obj:
  file_reader = reader(obj)
  header_line = next(file_reader)
  if header_line != None:
    for row in file_reader:  
      q_id = row[1]
      if not q_id in unique_problems:
        unique_problems.append(q_id)           

In [ ]:
len(unique_problems)

26688

In [ ]:
sample1 = open('sbnew.csv','r')
csv11 = csv.reader(sample1)
sort1 = sorted(csv11,key = operator.itemgetter(1))

In [ ]:
# Making a new csv file with sorted problem ids

file1 = open('sorted_problems.csv', 'w+', newline ='')
with file1:    
    write = csv.writer(file1)
    write.writerows(sort1)

In [ ]:
df3 = pd.read_csv('sorted_problems.csv', encoding = 'latin', low_memory = False)
print(df3.columns)
df3

Index(['user_id', 'problem_id', 'correct', 'attempt_count', 'answer_type',
       'type', 'skill_id', 'skill_name'],
      dtype='object')


,user_id,problem_id,correct,attempt_count,answer_type,type,skill_id,skill_name
0,74964,100522,1,1,choose_1,MasterySection,49.0,Ordering Positive Decimals
1,78093,100522,1,1,choose_1,MasterySection,49.0,Ordering Positive Decimals
2,78208,100522,1,1,choose_1,MasterySection,49.0,Ordering Positive Decimals
3,78266,100522,1,1,choose_1,MasterySection,49.0,Ordering Positive Decimals
4,78270,100522,0,1,choose_1,MasterySection,49.0,Ordering Positive Decimals
...,...,...,...,...,...,...,...,...
525529,78922,99710,1,1,choose_1,MasterySection,50.0,Ordering Fractions
525530,79056,99710,1,1,choose_1,MasterySection,50.0,Ordering Fractions
525531,96225,99710,1,1,choose_1,MasterySection,50.0,Ordering Fractions
525532,96230,99710,1,1,choose_1,MasterySection,50.0,Ordering Fractions


## id-skill map

In [7]:
skill_map = open('skillmap.csv','r')
csv112 = csv.reader(skill_map)
mycsv = list(csv112)
p_s_map = []
for i,row in enumerate(mycsv):
  if i ==0:
    continue
  p_id = row[0]
  s_id = row[1]
  if not p_id == "" and not s_id == "":
    pair = [p_id, s_id]
    p_s_map.append(pair)

In [8]:
len(p_s_map)

459208

In [9]:
# Remove duplicates so that we can build problem-skill map

p_s_map_ = []
for i in p_s_map:
    if i not in p_s_map_:
        p_s_map_.append(i)

In [10]:
len(p_s_map_)

21246

In [ ]:
p_s_map_

In [12]:
prob_2_index = {}   # Contains indices to the problem ids
prob_skill_map = {} # Dictionary with poblem ids as keys and values as skills for respective problems

skills_2_index = {} #Contains indices to the skills
skills = []         # Array containing all the unique skill ids

no_of_skills = 0    # Contains total no.of unique skills
no_of_probs = 0     # Contains total no.of unique problems

In [13]:
# Generating problem_id - skill mapping dictionary

#For every row in the previosly created p_s_map which contains problem ids and skill ids
for row in p_s_map_:
  problm = row[0]
  skill = row[1]
  
  # For every unique skill 
  if not problm in prob_skill_map:
    
    prob_skill_map[problm] = [skill]
    prob_2_index[problm] = no_of_probs

    #updating the no.of problems
    no_of_probs += 1

  else:  
    prob_skill_map[problm].append(skill)

    # For every unique skill 
  if not skill in skills_2_index:
    skills_2_index[skill] = no_of_skills 
    skills.append(skill)

    #updating the no.of skills
    no_of_skills += 1 

In [ ]:
prob_skill_map

In [ ]:
no_of_probs

17751

In [ ]:
no_of_skills

123

In [ ]:
len(prob_2_index)

17751

In [ ]:
len(skills)

123

In [ ]:
len(prob_skill_map)

17751

# Required files that were computed "student_data" & "prob_skill_map"

In [16]:
len(student_data)

4217

In [ ]:
prob_skill_map

In [ ]:
len(prob_skill_map)

17751

In [ ]:
student_data[0]

# Computations

## Test train split

In [20]:
set_size = float(len(student_data)) / 4
print ("Single set size: " + str(set_size))

Single set size: 1054.25


In [21]:
sets = []
for i in range(4):
  set = []

  for idx in range(int(math.floor(i * set_size)), int(math.floor((i + 1) * set_size))):
    set = set + student_data[idx]

  sets.append(set)  
  print ("Set" + str(i) +" contains "+ str(len(set))+ " records")

Set0 contains 305693 records
Set1 contains 62331 records
Set2 contains 47829 records
Set3 contains 109681 records


In [22]:
# Creating train-test split

for i in range(1):
  train_set = []
  test_set = []

  for j in range(4):
    if j != 2:
      for set in sets[j]:
        train_set.append(set)
    else:
      test_set = sets[j]

In [23]:
print("Size of Train set : " + str(len(train_set)))  
print("Size of Test set : " + str(len(test_set)))

Size of Train set : 477705
Size of Test set : 47829


In [ ]:
train_set

In [29]:
unique_stu = []
scores = [ [] for c in range(0, no_of_skills) ] #sublists equal to the number of skills where each sublist contains the scores obtained on that skill
stud_scores = [ [] for c in range(0, no_of_skills) ] #During iteration, this holds the scores obtained by a unique student


In [30]:
#Generating student scores from the training data

for r, row in enumerate(train_set):
  student = row[0]
  problm = row[1]

  # Checking if No.of attempts > 0, Otherwise we'll skip the iteration
  if float(row[3]) == 0:
    continue
  else:
    #score percent = No.of correctly answered/Total no. of Attempts to that question
    score_percent = int(round(float(row[2])/float(row[3]))) 

  if problm not in prob_skill_map:
    continue

  if not student in unique_stu:  
    unique_stu.append(student)
    for s in range(0, no_of_skills):
      if len(stud_scores[s]) > 0:
        scores[s].append(list(stud_scores[s])) 
    stud_scores = [ [] for c in range(0, no_of_skills) ] 

  for skill in prob_skill_map[problm]: 
    stud_scores[skills_2_index[skill]].append(score_percent)




In [31]:
# Appending the leftover data after last iteration to scores[]
for s in range(0, no_of_skills):
  if len(stud_scores[s]) > 0:
    scores[s].append(list(stud_scores[s]))

In [32]:
len(scores)

123

In [33]:
len(unique_stu)

3101

In [ ]:
print(scores.keys())

dict_keys(['14', '21825', '51950', '52613', '53167', '54318', '58161', '64525', '64531', '64532', '64535', '64550', '64634', '69931', '70190', '70363', '70657', '70676', '70677', '70679', '70682', '70684', '70687', '70688', '70692', '70693', '70695', '70699', '70701', '70702', '70704', '70706', '70707', '70708', '70709', '70727', '70728', '70729', '70730', '70731', '70732', '70733', '70734', '70736', '70740', '70743', '70744', '70745', '70746', '70748', '70750', '70752', '70754', '70756', '70758', '70759', '70816', '70872', '70874', '71034', '71066', '71068', '71105', '71113', '71144', '71146', '71148', '71151', '71152', '71163', '71173', '71174', '71175', '71176', '71177', '71179', '71180', '71181', '71183', '71185', '71187', '71188', '71190', '71191', '71192', '71193', '71198', '71203', '71205', '71208', '71209', '71210', '71211', '71212', '71213', '71215', '71216', '71219', '71221', '71224', '71225', '71227', '71230', '71231', '71232', '71234', '71235', '71250', '71427', '71557', '7

## Params

In [24]:
 # Initializing probability values to try
prob_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
probability_choices = []

for p_init in prob_values:
  for p_transit in prob_values:
    # We assume guess probability and slip probability will always be less than 0.5
    for p_guess in [x for x in prob_values if x < 0.5]: 
      for p_slip in [x for x in prob_values if x < 0.5]:
        probability_choices.append({'p_init': p_init,'p_transit': p_transit,'p_guess': p_guess, 'p_slip': p_slip})  

In [ ]:
'''import random
num = 100  # set the number to select here.
probability_choices_ = random.sample(probability_choices, num)'''



In [ ]:
#len(probability_choices_)

100

In [ ]:
#probability_choices =[]
#probability_choices = probability_choices_

In [25]:
len(probability_choices)

1296

In [26]:
probability_choices[0]

{'p_guess': 0.1, 'p_init': 0.1, 'p_slip': 0.1, 'p_transit': 0.1}

In [27]:
best_choice_ = [probability_choices[0] for x in range(0, no_of_skills)]
calculated_scores = [float("-inf") for x in range(0, no_of_skills)]
total_que = [0 for x in range(0, no_of_skills)]

In [ ]:
#---------------------------------------------------
# Finding the best probability parameters for each skill 
#--------------------------------------------------


#Iterating through the range of skills 
for i in range(0, no_of_skills):
  best_choice_[i] = probability_choices[0] #intializing with some random parameters, later gets the best parameters assigned at the end of loop

  # Iterate over range of all probability_choices
  for p, prob_parameter in enumerate(probability_choices):

    overall_score = 0
    no_of_attempts = 0

    # User if he solved a problem with this skill
    for u in range(0, len(scores[i])):
      user_score = 0

      # Intializing the probabilities
      p_mastered = prob_parameter['p_init']
      p_transit = prob_parameter['p_transit']
      p_slip = prob_parameter['p_slip']
                   
      # Initializing the compliments             
      p_guess_compliment = 1 - p_guess 
      p_slip_compliment = 1 - p_slip
      p_mastered_compliment = 1 - p_mastered

      no_of_attempts += len(scores[i][u]) 

      # Itererate over every score 
      for s in scores[i][u]: 

        p_correct = (p_mastered * p_slip_compliment) + (p_mastered_compliment * p_guess)
         
        user_score += math.log( (s * p_correct) - (( s - 1) * (1 - p_correct)) )

        x = ( s * p_mastered * p_slip_compliment / (p_mastered * p_slip_compliment + p_mastered_compliment * p_guess) )
        y = ( (s - 1) * p_mastered * p_slip / (p_mastered * p_slip + p_mastered_compliment * p_guess_compliment) )
        p_learn = x - y

        p_mastered = p_learn + (1 - p_learn) * p_transit
        p_mastered_compliment = 1 - p_mastered

      overall_score += user_score
    
    print(str(i) + "/" + str(no_of_skills))

    # Gathering the best parameters
    if overall_score > calculated_scores[i]:

      calculated_scores[i] = overall_score
      best_choice_[i] = probability_choices[p]
      total_que[i] = no_of_attempts




In [ ]:
# Storing the best parameters
best_choice = {} 

for c, skill in enumerate(skills):
  if c == 0:
    continue
  print(str(calculated_scores[c]) + "/" + str(total_que[c]))  
  best_choice[skill] = best_choice_[c]     
            

In [36]:
 #Creating a csv file with skills and their best params
 
 
 #Initiating the csv writer
 file_s = csv.writer(open('prob_params.csv', 'w', newline =''))

 # Inserting the columns into csv
 file_s.writerow(['skill', 'p_init', 'p_transit', 'p_guess', 'p_slip'])

 # Iteratively adding the rows { skill --> {params}}
 for skill_ in best_choice.keys():
   csv_row = [skill_, str(best_choice[skill_]['p_init']), str(best_choice[skill_]['p_transit']), str(best_choice[skill_]['p_guess']), str(best_choice[skill_]['p_slip'])]
   file_s.writerow(csv_row)
            

In [ ]:
best_choice

# Prediction

In [38]:
len(test_set)

47829

In [39]:
len(train_set)

477705

In [40]:
student_scores = [ {} for c in range(0, no_of_skills) ] 
student_lists = {}
unique = []

# Iterate over every row of test data
for r, row in enumerate(test_set):
  s_id = row[0]
  if float(row[3]) == 0: 
    continue                           # Not considering if the no.of attempts to the questions are zero
  else:
    score = int(round(float(row[2])/float(row[3])))  
  problm = row[1]                      # First column contains problem id
  
# Checking if the problem is in problem-skill-map
  if problm not in prob_skill_map:
    continue

# Checking is student id is in lists
  if s_id not in student_lists:
    student_lists[s_id] = {}            # creating student dict

  if problm not in  student_lists[s_id]:
    student_lists[s_id][problm] = []    # creating problem array for that student
    skill_indicies = []


# For every skill that student solves
  for skill in prob_skill_map[problm]:
                
    if s_id not in student_scores[skills_2_index[skill]]:
      student_scores[skills_2_index[skill]][s_id] = []

    student_scores[skills_2_index[skill]][s_id].append(score)
    skill_indicies.append(len(student_scores[skills_2_index[skill]][s_id])-1)


  student_lists[s_id][problm].append(skill_indicies) 
        

In [41]:
sores_n = [ {} for i in range(0, no_of_skills) ] 
      
# Iterate over every skill
for c, skill_s in enumerate(skills):

# Iteration over every user
  for sid in student_scores[c].keys():

    #checking if skill is already observed in best_choice
    if skill_s not in best_choice:
      best_choice[skill_s] = probability_choices[0] # Random assigning parameters

    # Initializing parameters with already calculated best parameters 
    p_mastered = best_choice[skill_s]['p_init']
    p_transit = best_choice[skill_s]['p_transit']
    p_guess = best_choice[skill_s]['p_guess']
    p_slip = best_choice[skill_s]['p_slip']
    

   # Initializing the complements
    p_slip_compliment = 1 - p_slip
    p_guess_compliment = 1 - p_guess
    p_mastered_compliment = 1 - p_mastered

    sores_n[c][sid] = []

    #Iterate through each scores            
    for s in student_scores[c][sid]:
      
      p_correct = (p_mastered * p_slip_compliment) + (p_mastered_compliment * p_guess)

      x = ( s * p_mastered * p_slip_compliment / (p_mastered * p_slip_compliment + p_mastered_compliment * p_guess) )
      y = ( (s - 1) * p_mastered * p_slip / (p_mastered * p_slip + p_mastered_compliment * p_guess_compliment) )
      p_learn = x - y

      p_mastered = p_learn + (1 - p_learn) * p_transit
      p_mastered_compliment = 1 - p_mastered

      sores_n[c][sid].append(p_correct)

                   
                

In [42]:
# Getting the predictions and scores
scores = {}
y = {}  

for s_id in student_lists:
  y[s_id] = []
  scores[s_id] = []
  for problm in student_lists[s_id]:
    for std_attmp in student_lists[s_id][problm]: 
      a_score = 0
      b_score = 0
      for i in range(0, len(std_attmp)):
        if i > len(prob_skill_map[problm])-1:
          continue
        index = std_attmp[i]  
        skill_i = skills_2_index[prob_skill_map[problm][i]]

        if index > len(sores_n[skill_i][s_id])-1:
          continue

        a_score += sores_n[skill_i][s_id][index]
        if i == 0:
          b_score = student_scores[skill_i][s_id][index]                 

      y[s_id].append(b_score)

      #Taking average of the score w.r.t the no.of skills per problem and inserting into final scores
      scores[s_id].append(a_score/len(prob_skill_map[problm]))

In [ ]:
scores

In [ ]:
#Writing the results into csv

file_w = csv.writer(open('results.csv', 'w', newline =''))
file_w.writerow(['y', 'prediction'])
for s_id in y.keys(): 
  for i in range(0, len(y[s_id])):
    entry = [str(y[s_id][i]), str(scores[s_id][i])]
    file_w.writerow(entry)
  file_w.writerow([])

# AUC

In [47]:
file_result = open('results.csv', 'r')
  
# writing the data into the file
with file_result:    
    read = csv.reader(file_result)
    value_s = []
    score_s = []
    for i,row in enumerate(read):
      if i ==0:
        continue
      if len(row) is not 0:
        value_s.append(int(row[0]))
        score_s.append(float(row[1]))
    values = np.array(value_s)
    scores = np.array(score_s)

auc = roc_auc_score(values, scores)
print ("AUC score : "+ str(auc))

AUC score : 0.6917493465741728


# ------------------------------------------------------------------------------------------------------------------